# Cleaning Oscars Awards Data
**Author:** Mackenzie Ross

**Date Last Modified:** 31 March 2023

**Description:** Preparing the Oscars awards dataset to be used in a SQL database

## Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np

## Load and Preview Dataset

In [2]:
df = pd.read_csv('raw_oscars_data.csv')
df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10765 entries, 0 to 10764
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10765 non-null  int64 
 1   year_ceremony  10765 non-null  int64 
 2   ceremony       10765 non-null  int64 
 3   category       10765 non-null  object
 4   name           10761 non-null  object
 5   film           10450 non-null  object
 6   winner         10765 non-null  bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 515.2+ KB


## Investigate Null Values

In [4]:
# print number of rows with null name value
missing_names = pd.isnull(df['name'])
df[missing_names]

,year_film,year_ceremony,ceremony,category,name,film,winner
10513,2020,2021,93,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10514,2020,2021,93,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10635,2021,2022,94,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10759,2022,2023,95,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True


In [5]:
# print number of rows with null film value
missing_film = pd.isnull(df['film'])
df[missing_film]

,year_film,year_ceremony,ceremony,category,name,film,winner
16,1927,1928,1,ENGINEERING EFFECTS,Ralph Hammeras,NaN,False
18,1927,1928,1,ENGINEERING EFFECTS,Nugent Slaughter,NaN,False
31,1927,1928,1,WRITING (Title Writing),Joseph Farnham,NaN,True
32,1927,1928,1,WRITING (Title Writing),"George Marion, Jr.",NaN,False
33,1927,1928,1,SPECIAL AWARD,Warner Bros.,NaN,True
...,...,...,...,...,...,...,...
10759,2022,2023,95,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10760,2022,2023,95,HONORARY AWARD,"To Euzhan Palcy, a masterful filmmaker who bro...",NaN,True
10761,2022,2023,95,HONORARY AWARD,"To Diane Warren, for her genius, generosity an...",NaN,True
10762,2022,2023,95,HONORARY AWARD,"To Peter Weir, a fearless and consummate filmm...",NaN,True


There are 4 rows missing with a NULL value in name and 315 rows with a null value in film. The rows missing a film title are primarily defunt and honorary awards. We will drop all the rows with a null value. All of the rows with missing names are also missing the film title.

In [6]:
# drop rows with NULL value
df1 = df.dropna(axis = 0, how ='any').copy()

# comparing sizes of data frames
print("Old data frame length:", len(df), "\nNew data frame length:",
	len(df1), "\nNumber of rows with at least 1 NA value: ")

Old data frame length: 10765 
New data frame length: 10450 
Number of rows with at least 1 NA value: 


## Refine nomination categories

In [7]:
# change category names to lower case
df1['category'] = [word.lower() for word in df1['category']]

In [8]:
# get unique nomination categories
df1['category'].value_counts(ascending = False).head(20)

directing                       464
film editing                    445
actress in a supporting role    435
actor in a supporting role      435
documentary (short subject)     378
best picture                    361
documentary (feature)           345
cinematography                  333
foreign language film           315
art direction                   307
costume design                  290
music (original score)          265
sound                           240
actress                         236
actress in a leading role       235
actor in a leading role         235
actor                           232
music (original song)           230
short film (live action)        221
music (song)                    215
Name: category, dtype: int64

Special Achievement Awards and Awards of Commendation are not given out annually, so we want to drop those rows.

In [9]:
df2 = df1[df1['category'].str.contains('special achievement|award') == False].copy()

## Clean data

In [10]:
# remove unnecessary parentheses from cells in name column
df2['name'] = df2['name'].str.replace(r'[()]','', regex=True)

In [11]:
# remove unnecessary semicolons from cells in film column
df2['film'] = df2['film'].str.replace(';','', regex=True)

In [12]:
# split cells that have multiple values in the name column (some have commas, some have semi-colons)
df3 = df2.set_index(['year_film', 'year_ceremony', 'ceremony', 'category', 'film', 'winner']).apply(lambda x: x.str.split(',').explode()).reset_index()
df4 = df3.set_index(['year_film', 'year_ceremony', 'ceremony', 'category', 'film', 'winner']).apply(lambda x: x.str.split(';').explode()).reset_index()
df5 = df4.set_index(['year_film', 'year_ceremony', 'ceremony', 'category', 'film', 'winner']).apply(lambda x: x.str.split('&').explode()).reset_index()
df6 = df5.set_index(['year_film', 'year_ceremony', 'ceremony', 'category', 'film', 'winner']).apply(lambda x: x.str.split(' and ').explode()).reset_index()
# remove leading/trailing whitespace
df6['name'] = df6['name'].str.strip()

print('New length of data frame:', len(df6))

New length of data frame: 17750


Some names are listed with their person's title listed after their name, so we need to identify these titles an remove those rows from the data frame.

In [13]:
df6['name'].value_counts(ascending = False).head(15)

Producer                                       824
Producers                                      428
Sound Director                                 236
Music                                          181
Jr.                                             66
Metro-Goldwyn-Mayer                             64
Walt Disney                                     60
John Williams                                   45
Alfred Newman                                   43
Warner Bros.                                    43
France                                          38
Edith Head                                      35
Italy                                           29
Art Direction:  Cedric Gibbons                  28
Metro-Goldwyn-Mayer Studio Sound Department     26
Name: name, dtype: int64

In [14]:
name_to_remove = ['Producer', 'Producers', 'Sound Director']

In [15]:
# remove specified columns
df7 = df6[df6.name.isin(name_to_remove) == False].copy()

In [16]:
# check that names were removed
df7['name'].value_counts(ascending = False).head()

Music                  181
Jr.                     66
Metro-Goldwyn-Mayer     64
Walt Disney             60
John Williams           45
Name: name, dtype: int64

Some columns start with position titles such as"Art Direction:", "Lyrics by", or "Music by". We want to remove this title from those cells and keep the person's name only.

In [17]:
df7['name'] = df7['name'].str.replace('Art Direction:', '')
df7['name'] = df7['name'].str.replace('Lyrics by', '')
df7['name'] = df7['name'].str.replace('Music by', '')
df7['name'] = df7['name'].str.replace('Written by', '')
df7['name'] = df7['name'].str.replace('Story by', '')
df7['name'] = df7['name'].str.replace('Score by', '')
df7['name'] = df7['name'].str.replace('head of department', '')
df7['name'] = df7['name'].str.replace('Lyric by', '')
df7['name'] = df7['name'].str.replace('Screenplay by', '')
df7['name'] = df7['name'].str.replace('Adaptation by', '')
df7['name'] = df7['name'].str.replace('Written for the screen by', '')
df7['name'] = df7['name'].str.replace('Original story by', '')
df7['name'] = df7['name'].str.replace('Thematic', '')
df7['name'] = df7['name'].str.strip()

In [18]:
df7[df7['name'] == 'Jr.']

,year_film,year_ceremony,ceremony,category,film,winner,name
157,1930,1931,4,writing (adaptation),The Criminal Code,False,Jr.
232,1932,1933,6,cinematography,A Farewell to Arms,True,Jr.
707,1937,1938,10,assistant director,Lost Horizon,False,Jr.
1284,1940,1941,13,cinematography (black-and-white),"Arise, My Love",False,Jr.
1839,1942,1943,15,documentary,Conquer by the Clock,False,Jr.
...,...,...,...,...,...,...,...
14398,2008,2009,81,makeup,The Dark Knight,False,Jr.
15814,2014,2015,87,writing (original screenplay),Birdman or (The Unexpected Virtue of Ignorance),True,Jr.
16446,2017,2018,90,short film (live action),My Nephew Emmett,False,Jr.
16998,2020,2021,93,actor in a supporting role,One Night in Miami...,False,Jr.


Lastly, we want to reunite the "Jr." suffixes that were separated from the names when we split the name column on commas.

In [19]:
# get list of all rows with a Jr. value in the name column
jr_list = df7.index[df7['name'] == 'Jr.'].tolist()
print(jr_list)

[157, 232, 707, 1284, 1839, 2029, 2251, 3062, 3086, 3230, 3704, 3951, 3962, 4289, 4302, 4369, 4700, 5043, 5145, 5335, 5529, 5723, 5954, 6102, 6118, 6150, 6165, 6248, 6254, 6622, 7025, 7109, 7541, 7620, 7666, 7765, 7925, 7960, 8391, 8398, 8476, 8669, 8937, 9174, 9357, 9378, 9488, 9529, 10008, 10081, 10084, 10513, 11102, 11347, 11519, 11529, 12023, 12081, 12112, 13281, 13494, 14398, 15814, 16446, 16998, 17113]


In [20]:
# append Jr. to appropriate name values
for i in jr_list:
    j = i - 1
    df7.loc[j, 'name'] = df7.loc[j, 'name'] + ' Jr.'

In [21]:
# remove rows with just Jr. in name column
oscars = df7.drop(jr_list)

In [22]:
oscars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16196 entries, 0 to 17749
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      16196 non-null  int64 
 1   year_ceremony  16196 non-null  int64 
 2   ceremony       16196 non-null  int64 
 3   category       16196 non-null  object
 4   film           16196 non-null  object
 5   winner         16196 non-null  bool  
 6   name           16196 non-null  object
dtypes: bool(1), int64(3), object(3)
memory usage: 901.5+ KB


## Save new dataset as CSV file

In [23]:
oscars.to_csv('oscars_cleaned.csv')